In [ ]:
%load_ext autoreload
import numpy as np
import h5py as h5
import pandas as pd
import matplotlib.pyplot as plt
import scqubits as scq
import plotly.graph_objects as go

%autoreload 2
from zcu_tools.analysis.fluxdep import calculate_energy, VisualizeSpet, load_result

In [ ]:
qub_name = "Test096"

# Load data

## Parameters

In [ ]:
loadpath = f"../result/{qub_name}/params.json"
_, params, cflx, period, allows = load_result(loadpath)
EJ, EC, EL = params

if "r_f" in allows:
    r_f = allows["r_f"]


def phi2mA(ratio):
    return (period * (ratio - 0.5) + cflx) * 1e3


def mA2phi(current):
    return (current / 1e3 - cflx) / period + 0.5


flxs = np.linspace(0.0, 1.0, 10000)
mAs = phi2mA(flxs)

## Sample Points

In [ ]:
# loading points
loadpath = f"../result/{qub_name}/freqs.csv"

freqs_df = pd.read_csv(loadpath)
s_mAs = freqs_df["Current (mA)"].values  # mA
s_fpts = freqs_df["Freq (MHz)"].values * 1e-3  # GHz
s_T1s = freqs_df["T1 (μs)"].values
s_T1errs = freqs_df["T1err (μs)"].values

# sort by flux
s_mAs, s_fpts, s_T1s, s_T1errs = tuple(
    np.array(a) for a in zip(*sorted(zip(s_mAs, s_fpts, s_T1s, s_T1errs)))
)

s_flxs = mA2phi(s_mAs)

freqs_df.head(30)

## TwoTone Spectrum

In [ ]:
load_path = f"../result/{qub_name}/spect.hdf5"

s_spects = {}
with h5.File(load_path, "r") as f:
    for key in f.keys():
        grp = f[key]
        spectrum = (
            grp["spectrum"]["mAs"][()],
            grp["spectrum"]["fpts"][()],
            grp["spectrum"]["data"][()],
        )
        points = (grp["points"]["mAs"][()], grp["points"]["fpts"][()])
        s_spects.update(
            {
                key: {
                    "mA_c": grp["mA_c"][()] * 1e3,
                    "spectrum": spectrum,
                    "points": points,
                }
            }
        )
s_spects.keys()

### Align cflx

In [ ]:
mA_c = list(s_spects.values())[-1]["mA_c"]
for spect in s_spects.values():
    shift = mA_c - spect["mA_c"]
    spect["mA_c"] += shift
    spect["spectrum"] = (phi2mA(spect["spectrum"][0] + shift), *spect["spectrum"][1:])
    spect["points"] = (phi2mA(spect["points"][0] + shift), *spect["points"][1:])

# Plot Map

In [ ]:
r_f = 7.48742
allows = {
    **allows,
    # "transitions": [(0, 1)],
    # "transitions": [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)],
    # "red side": [(0, 4), (0, 3), (1, 5)],
    # "r_f": r_f,
}

In [ ]:
energies = calculate_energy(flxs, *params, 50, 15)

In [ ]:
vs = VisualizeSpet(s_spects, s_mAs, s_fpts, mAs, energies, allows)
vs.set_scatter_style(10, color=s_T1s)
fig = vs.create_figure()
fig.add_hline(y=r_f, line_dash="dash", line_color="black", name="Cavity freq")
fig.add_vline(x=cflx * 1e3, line_dash="dash", line_color="black", name="Cavity freq")
fig.update_layout(
    title_text=f"EJ/EC/EL = {EJ:.2f}/{EC:.2f}/{EL:.2f}",
    showlegend=True,
)

fig.show()

## save to html

In [ ]:
fig.write_html(f"../result/{qub_name}/web/map.html", include_plotlyjs="cdn")
fig.write_image(f"../result/{qub_name}/image/map.png", format="png")

# T1 curve

In [ ]:
fig, ax1 = plt.subplots(constrained_layout=True)

ax1.errorbar(s_mAs, s_T1s, yerr=s_T1errs, fmt=".-", label="Current")
ax1.grid()
ax1.set_xlabel(r"Current (mA)")
ax1.set_ylabel(r"$T_1$ ($\mu s$)")
ax2 = ax1.secondary_xaxis("top", functions=(mA2phi, phi2mA))
ax2.set_xlabel(r"$\phi_{ext}/\phi_0$")

fig.savefig(f"../result/{qub_name}/image/T1s.png")
plt.show()

In [ ]:
Temp = 54.2e-3
Q_ind = 6.5e5
Q_cap = 7e4
x_qp = 9e-6
flxs = np.linspace(s_flxs.min(), s_flxs.max(), 501)

scq.settings.T1_DEFAULT_WARNING = False
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=30, truncated_dim=5)
T1_eff_fig, T1_eff_ax = fluxonium.plot_t1_effective_vs_paramvals(
    param_name="flux",
    param_vals=flxs,
    xlim=([flxs.min(), flxs.max()]),
    common_noise_options=dict(i=1, j=0, T=Temp),
    noise_channels=[
        ("t1_inductive", dict(Q_ind=Q_ind)),
        ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
        ("t1_capacitive", dict(Q_cap=Q_cap)),
    ],
)

T1_eff_ax.errorbar(s_flxs, s_T1s * 1e3, yerr=s_T1errs * 1e3, fmt=".-")
T1_eff_ax.set_ylim([s_T1s.min() * 5e2, s_T1s.max() * 2e3])
_, T1_eff = T1_eff_ax.lines[0].get_data()
plt.show()

In [ ]:
T1_cap = []
T1_ind = []
T1_qp = []
for flux in flxs:
    fluxonium.flux = flux
    T1_cap.append(fluxonium.t1_capacitive(T=Temp, Q_cap=Q_cap) * 1e-9)
    T1_ind.append(fluxonium.t1_inductive(T=Temp, Q_ind=Q_ind) * 1e-9)
    T1_qp.append(fluxonium.t1_quasiparticle_tunneling(T=Temp, x_qp=x_qp) * 1e-9)

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(flxs, T1_cap, label="Capacitive")
ax.plot(flxs, T1_ind, label="Inductive")
ax.plot(flxs, T1_qp, label="Quasi particle")
ax.plot(flxs, T1_eff * 1e-9, label="Effective")
ax.errorbar(
    s_flxs, s_T1s * 1e-6, yerr=s_T1errs * 1e-6, fmt=".-", label="Data", color="purple"
)
ax.set_xlabel(r"$\phi_{ext}/\phi_0$")
ax.set_ylabel(r"$T_1$ (s)")
ax.set_ylim([s_T1s.min() * 5e-7, s_T1s.max() * 2e-5])
ax.set_title(f"Q_cap = {Q_cap:.1e}, Q_ind = {Q_ind:.1e}, x_qp = {x_qp:.1e}")
ax.grid()
ax.legend()
plt.tight_layout()
fig.savefig(f"../result/{qub_name}/image/T1_fit.png")
plt.show()

# Matrix elements

In [ ]:
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=50, truncated_dim=15)
spectrumData = fluxonium.get_matelements_vs_paramvals(
    operator="n_operator", param_name="flux", param_vals=flxs, evals_count=10
)
matrixelements = spectrumData.matrixelem_table

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(5) if j > i]

fig = go.Figure()
for i, (from_, to_) in enumerate(show_idxs):
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=np.abs(matrixelements[:, from_, to_]),
            mode="lines",
            name=f"{from_}-{to_}",
            line=dict(width=2),
        )
    )
fig.update_layout(
    title=f"EJ/EC/EL = {EJ:.3f}/{EC:.3f}/{EL:.3f}",
    title_x=0.5,
    xaxis_title=r"$\phi_{ext}/\phi_0$",
    yaxis_title="Matrix elements",
)

fig.show()

In [ ]:
fig.write_html(f"../result/{qub_name}/web/matrixelem.html", include_plotlyjs="cdn")
fig.write_image(f"../result/{qub_name}/image/matrixelem.png", format="png")